In [185]:
import email, smtplib, ssl
import sys
import pandas as pd
import numpy as np
import os
from datetime import datetime
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime
from google.colab import drive
from string import Template

# En esta etapa se preparan los archivos base y el archivo de concentrados

In [2]:
pd.set_option('display.float_format', '{:,.2f}'.format)

In [3]:
pk_for_files =datetime.now().strftime("%d/%m/%Y %H:%M:%S").replace("/","").replace(" ","").replace(":","") #esto deberia estar cerca de la cabecera
pk_for_files

'31082020043737'

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd '/content/drive/My Drive/Medica_Vial_Experiment_1p1/input_files'
!ls

/content/drive/My Drive/Medica_Vial_Experiment_1p1/input_files
 base_files_submitted   providers
 concentrado.xlsx      'PRUEBA 08242020 SOBRE FOLIOS YA PAGADOS.xlsx'


In [6]:
basefile_name = os.listdir()        #https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory#3207973
basefile_name [0]

'PRUEBA 08242020 SOBRE FOLIOS YA PAGADOS.xlsx'

In [7]:
basefile = pd.read_excel(basefile_name [0], index = False)
basefile.rename( columns = {'Tabulador':'Subtotal'}, inplace = True)
basefile

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,Consecutivo
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",300
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",300
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",300
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",300
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",300
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300


In [8]:
conc_file = pd.read_excel('concentrado.xlsx', index = False)
conc_file.rename( columns = {'ClaveInterna':'Clave'}, inplace = True)
conc_file

,ID,Clave,Folio,Estatus
0,2,NLTHMA,TAMA001367,PENDIENTE
1,3,NLTHMA,TAMA001369,PENDIENTE
2,4,NLTHMA,TAMA001380,PENDIENTE
3,5,NLTHMA,TAMA001381,PENDIENTE
4,6,NLTHMA,TAMA001383,PENDIENTE
5,7,NLTHMA,TAMA001384,PENDIENTE
6,8,NLTHMA,TAMA001388,PENDIENTE
7,9,NLTHMA,TAMA001389,PENDIENTE
8,10,NLTHMA,TAMA001391,PENDIENTE
9,11,NLTHMA,TAMA001393,PENDIENTE


In [9]:
clavesIVAFrontera = ["TACCMF", "OAXMED", "TIJHVI", "ENSSRL","REYSAN", "NLTHMA"]

# Los indices se ajustan antes para hacer la concatenación interna

In [10]:
conc_file = conc_file.set_index(['Clave','Folio']) #basefile
conc_file

ID    Estatus
Clave  Folio                    
NLTHMA TAMA001367   2  PENDIENTE
       TAMA001369   3  PENDIENTE
       TAMA001380   4  PENDIENTE
       TAMA001381   5  PENDIENTE
       TAMA001383   6  PENDIENTE
       TAMA001384   7  PENDIENTE
       TAMA001388   8  PENDIENTE
       TAMA001389   9  PENDIENTE
       TAMA001391  10  PENDIENTE
       TAMA001393  11  PENDIENTE
       TAMA001394  12  PENDIENTE
MORLUZ LUMO001603  13  PENDIENTE
       LUMO001604  14  PENDIENTE
       LUMO001605  15  PENDIENTE
       LUMO001606  16  PENDIENTE
       LUMO001607  17  PENDIENTE
       LUMO001608  18  PENDIENTE
       LUMO001609  19  PENDIENTE
LNHMBR LEBR000286  20  PENDIENTE
       LEBR000287  21  PENDIENTE
       LEBR000287  22  PENDIENTE
       LEBR000287  23  PENDIENTE
       LEBR000300  24  PENDIENTE
TLXHSM TLSM002828  25  PENDIENTE
       TLSM002837  26  PENDIENTE
       TLSM002856  27  PENDIENTE
       TLSM002872  28  PENDIENTE
       TLSM002872  29  PENDIENTE
       TLSM002873  30  PENDIENTE
       TLSM002873  31  PENDIENTE

In [11]:
basefile = basefile.set_index(['Clave','Folio']) #basefile
basefile

Unidad  ... Consecutivo
Clave  Folio                                    ...            
NLTHMA TAMA001367  NL Tamps-H. MexicoAmericano  ...         300
       TAMA001369  NL Tamps-H. MexicoAmericano  ...         300
       TAMA001380  NL Tamps-H. MexicoAmericano  ...         300
       TAMA001381  NL Tamps-H. MexicoAmericano  ...         300
       TAMA001383  NL Tamps-H. MexicoAmericano  ...         300
       TAMA001384  NL Tamps-H. MexicoAmericano  ...         300
       TAMA001388  NL Tamps-H. MexicoAmericano  ...         300
       TAMA001389  NL Tamps-H. MexicoAmericano  ...         300
       TAMA001391  NL Tamps-H. MexicoAmericano  ...         300
       TAMA001393  NL Tamps-H. MexicoAmericano  ...         300
       TAMA001394  NL Tamps-H. MexicoAmericano  ...         300
MORLUZ LUMO001603               Morelia-La luz  ...         150
       LUMO001604               Morelia-La luz  ...         150
       LUMO001605               Morelia-La luz  ...         150
       LUMO001606               Morelia-La luz  ...         150
       LUMO001607               Morelia-La luz  ...         150
       LUMO001608               Morelia-La luz  ...         150
       LUMO001609               Morelia-La luz  ...         150
LNHMBR LEBR000286               LeÃ³n-H Brisas  ...         210
       LEBR000287               LeÃ³n-H Brisas  ...         210
       LEBR000287               LeÃ³n-H Brisas  ...         210
       LEBR000287               LeÃ³n-H Brisas  ...         210
       LEBR000300               LeÃ³n-H Brisas  ...         210
TLXHSM TLSM002828           Tlax-H. San Martin  ...         530
       TLSM002837           Tlax-H. San Martin  ...         530
       TLSM002856           Tlax-H. San Martin  ...         530
       TLSM002872           Tlax-H. San Martin  ...         530
       TLSM002872           Tlax-H. San Martin  ...         530
       TLSM002873           Tlax-H. San Martin  ...         530
       TLSM002873           Tlax-H. San Martin  ...         530

[30 rows x 8 columns]

# Solo se consideran todos los datos que comparten los mismos indices. Posteriormente se reinician los indices.
## Nota mental: Tal vez el problema de concetrados se pueda solucionar con la pregunta ¿De donde sale el archivo base?

In [12]:
MV_first_output_raw = pd.concat([conc_file, basefile], axis = 1, join = 'inner') #pd.concat([df_1, df_2], axis = 1, join = 'inner') 
MV_first_output_raw = MV_first_output_raw.reset_index( drop = False)

# Se limpian las columas necesarias para el primer output

In [13]:
#list(MV_first_output_raw)
MV_first_output = MV_first_output_raw[['Clave','Unidad','Folio','Lesionado','Etapa', 'Entrega','Tipo', 'Lesion', 'Subtotal', 'Consecutivo', 'Estatus']]
MV_first_output

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,Consecutivo,Estatus
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",300,PENDIENTE
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",300,PENDIENTE
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",300,PENDIENTE
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",300,PENDIENTE
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",300,PENDIENTE
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE


# Esta función lambda me permite definir a los servicios pendientes como verdaderos y a los demas como falsos


In [14]:
MV_first_output[ 'Bool_Status' ] = MV_first_output[ 'Estatus' ].apply( lambda x: True if x == 'PENDIENTE' else False)
MV_first_output

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,Consecutivo,Estatus,Bool_Status
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",300,PENDIENTE,True
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",300,PENDIENTE,True
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",300,PENDIENTE,True
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",300,PENDIENTE,True
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",300,PENDIENTE,True
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True


# A continuación, solo filtraré aquellos valores verdaderos.

In [15]:
MV_first_output.loc[ MV_first_output.Bool_Status ]

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,Consecutivo,Estatus,Bool_Status
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",300,PENDIENTE,True
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",300,PENDIENTE,True
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",300,PENDIENTE,True
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",300,PENDIENTE,True
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",300,PENDIENTE,True
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True


In [16]:
MV_first_output = MV_first_output.loc[ MV_first_output.Bool_Status ].reset_index( drop = True)
MV_first_output 

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,Consecutivo,Estatus,Bool_Status
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",300,PENDIENTE,True
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",300,PENDIENTE,True
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",300,PENDIENTE,True
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",300,PENDIENTE,True
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",300,PENDIENTE,True
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True


# Esta función lambda me permite distinguir las claves frontera de las que no lo son

In [17]:
MV_first_output['IVA_rate'] = MV_first_output[ 'Clave' ].apply( lambda x: 0.08 if x in clavesIVAFrontera  else 0.16)
MV_first_output

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,Consecutivo,Estatus,Bool_Status,IVA_rate
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",300,PENDIENTE,True,0.08
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",300,PENDIENTE,True,0.08
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True,0.08
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True,0.08
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",300,PENDIENTE,True,0.08
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True,0.08
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",300,PENDIENTE,True,0.08
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",300,PENDIENTE,True,0.08
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True,0.08
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True,0.08


# Se realizan operaciones y se crean nuevas columnas con base en las columnas que ya existen

In [18]:
MV_first_output['IVA'] = MV_first_output['Subtotal'] * MV_first_output['IVA_rate']
MV_first_output

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,Consecutivo,Estatus,Bool_Status,IVA_rate,IVA
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",300,PENDIENTE,True,0.08,92.40
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",300,PENDIENTE,True,0.08,92.40
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True,0.08,92.40
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True,0.08,92.40
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",300,PENDIENTE,True,0.08,92.40
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True,0.08,92.40
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",300,PENDIENTE,True,0.08,92.40
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",300,PENDIENTE,True,0.08,92.40
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True,0.08,92.40
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True,0.08,92.40


# Se adiciona la columna ones para conteo de servicios.

In [19]:
MV_first_output['Total'] = MV_first_output['Subtotal'] + MV_first_output['IVA']
MV_first_output['Ones'] = 1
MV_first_output

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,Consecutivo,Estatus,Bool_Status,IVA_rate,IVA,Total,Ones
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",300,PENDIENTE,True,0.08,92.40,"1,247.40",1
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",300,PENDIENTE,True,0.08,92.40,"1,247.40",1
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True,0.08,92.40,"1,247.40",1
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True,0.08,92.40,"1,247.40",1
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",300,PENDIENTE,True,0.08,92.40,"1,247.40",1
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True,0.08,92.40,"1,247.40",1
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",300,PENDIENTE,True,0.08,92.40,"1,247.40",1
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",300,PENDIENTE,True,0.08,92.40,"1,247.40",1
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",300,PENDIENTE,True,0.08,92.40,"1,247.40",1
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",300,PENDIENTE,True,0.08,92.40,"1,247.40",1


# La siguiente es la filtración definitiva

In [20]:
#list(MV_first_output)
MV_first_output = MV_first_output[['Clave','Unidad','Folio','Lesionado','Etapa', 'Entrega','Tipo', 'Lesion', 'Subtotal', 'IVA', 'Total', 'Consecutivo', 'Ones']]
MV_first_output

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,IVA,Total,Consecutivo,Ones
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",92.40,"1,247.40",300,1
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",92.40,"1,247.40",300,1
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",92.40,"1,247.40",300,1
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",92.40,"1,247.40",300,1
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",92.40,"1,247.40",300,1
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",92.40,"1,247.40",300,1
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",92.40,"1,247.40",300,1
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",92.40,"1,247.40",300,1
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",92.40,"1,247.40",300,1
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",92.40,"1,247.40",300,1


In [21]:
%cd '/content/drive/My Drive/Medica_Vial_Experiment_1p1/output_files/Reports'

/content/drive/My Drive/Medica_Vial_Experiment_1p1/output_files/Reports


# Voy a crear mi primer archivo de salida, con un nombre único

In [22]:
MV_first_output.to_excel(f'MedicaVial_Report_{pk_for_files}.xlsx', index = False)

# Agrupo por clave y por etapa, con el fin de calcular el subtotal, el iva y el total de cada factura.

In [23]:
MV_Subtotal_Factura = MV_first_output.groupby(['Clave','Etapa'])['Subtotal'].sum().to_frame()
MV_Subtotal_Factura

Subtotal
Clave  Etapa          
LNHMBR 2        920.00
       3      2,180.00
MORLUZ 1      7,100.00
NLTHMA 1     12,705.00
TLXHSM 2        810.00
       3      4,013.33

In [24]:
MV_Subtotal_Factura = MV_Subtotal_Factura.reset_index( drop = False )

In [25]:
MV_Subtotal_Factura

,Clave,Etapa,Subtotal
0,LNHMBR,2,920.00
1,LNHMBR,3,"2,180.00"
2,MORLUZ,1,"7,100.00"
3,NLTHMA,1,"12,705.00"
4,TLXHSM,2,810.00
5,TLXHSM,3,"4,013.33"


In [26]:
MV_IVA_Factura = MV_first_output.groupby(['Clave','Etapa'])['IVA'].sum().to_frame()
MV_IVA_Factura 

IVA
Clave  Etapa         
LNHMBR 2       147.20
       3       348.80
MORLUZ 1     1,136.00
NLTHMA 1     1,016.40
TLXHSM 2       129.60
       3       642.13

In [27]:
MV_IVA_Factura = MV_IVA_Factura.reset_index( drop = False)
MV_IVA_Factura

,Clave,Etapa,IVA
0,LNHMBR,2,147.20
1,LNHMBR,3,348.80
2,MORLUZ,1,"1,136.00"
3,NLTHMA,1,"1,016.40"
4,TLXHSM,2,129.60
5,TLXHSM,3,642.13


In [28]:
MV_Total_Atenciones = MV_first_output.groupby(['Clave','Etapa'])['Ones'].sum().to_frame()
MV_Total_Atenciones = MV_Total_Atenciones.reset_index( drop = False)
MV_Total_Atenciones.rename( columns = {'Ones':'Total de atenciones'}, inplace = True)
MV_Total_Atenciones

,Clave,Etapa,Total de atenciones
0,LNHMBR,2,3
1,LNHMBR,3,2
2,MORLUZ,1,7
3,NLTHMA,1,11
4,TLXHSM,2,2
5,TLXHSM,3,5


In [29]:
MV_Total_Factura = MV_first_output.groupby(['Clave','Etapa'])['Total'].sum().to_frame()
MV_Total_Factura = MV_Total_Factura.reset_index( drop = False)
MV_Total_Factura

,Clave,Etapa,Total
0,LNHMBR,2,"1,067.20"
1,LNHMBR,3,"2,528.80"
2,MORLUZ,1,"8,236.00"
3,NLTHMA,1,"13,721.40"
4,TLXHSM,2,939.60
5,TLXHSM,3,"4,655.46"


In [30]:
#27/08/2020 Ahora que tengo todos los totales separados por clave y por etapas, toca construir una única tabla, despues debemos separar cada tabla por unidad médica.

# Queda pendiente la union de todos los dataframes anteriores. Ahora preparo el segundo archivo de salida, antes de enviar los mensajes por correo electrónico.

In [31]:
%cd '/content/drive/My Drive/Medica_Vial_Experiment_1p1/input_files/providers'
!ls
providers_dir = pd.read_excel('BASEPROVEEDORES_Test.xlsx', index = False)
providers_dir

/content/drive/My Drive/Medica_Vial_Experiment_1p1/input_files/providers
BASEPROVEEDORES_Test.xlsx  BASEPROVEEDORES.xlsx


,Clave,RAZON SOCIAL,RFC,Email,Persona,MAIL SOLICITUD DE FACTURAS
0,AGAMAC,"Centro Hospitalario MAC, S.A. de C.V.",CHO0801174Z5,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
1,CANAVC,"AVCP, S.A. de C.V.",AVC161128RH6,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
2,CANHAC,"Hospital México Americano del Caribe, S.A. de ...",HMA160502HW4,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
3,CELMAC,"Centro Hospitalario MAC, S.A. de C.V.",CHO0801174Z5,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
4,CELSCE,"Sanatorio Celaya, S.A. de C.V.",SCE8410082G5,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
...,...,...,...,...,...,...
64,VERSLJ,"slj Medical, S. de R.L. de C.V.",SME120709340,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
65,XALHVI,Javier Armando Arreguin Ruiz,AERJ560801J46,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",F,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
66,ZACHSE,"Especialidades Medicas Santa Elena, S.A. de C.V.",EMS900613HW9,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
67,ZAPARB,"Sante Medical Center, S. de C.V.",SMC0409234D5,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."


In [32]:
df_1 = MV_first_output.set_index(['Clave'])
df_2 = providers_dir.set_index(['Clave'])
MV_second_output_raw = pd.concat([df_1, df_2], axis = 1, join = 'inner')
MV_second_output_raw = MV_second_output_raw.reset_index( drop = False)
MV_second_output_raw

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,IVA,Total,Consecutivo,Ones,RAZON SOCIAL,RFC,Email,Persona,MAIL SOLICITUD DE FACTURAS
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail....",M,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."


In [33]:
#list(MV_first_output)
MV_first_output = MV_first_output[['Clave','Folio','Lesionado','Etapa', 'Entrega','Tipo', 'Lesion', 'Subtotal', 'IVA', 'Total']]

In [34]:
Set_claves = list(set(list(MV_first_output['Clave'])))
Set_claves

['LNHMBR', 'NLTHMA', 'TLXHSM', 'MORLUZ']

In [37]:
#Esta linea es solo un ensayo, no es necesario que se agrege al script final.
Set_etapas = list(set(list(MV_first_output['Etapa'])))
Set_etapas

[1, 2, 3]

In [38]:
#Esta linea es solo un ensayo, no es necesario que se agrege al script final.
gapminder = MV_first_output[MV_first_output.Clave.eq('LNHMBR')]
gapminder_e = gapminder[gapminder.Etapa.eq(2)]
gapminder_e

,Clave,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,IVA,Total
18,LNHMBR,LEBR000286,RAFAEL DE LA TORRE GUTIERREZ,2,1,GestiÃ³n,Paquete TTL,200.00,32.00,232.00
19,LNHMBR,LEBR000287,CLAUDIA CASTILLO OROZCO,2,1,GestiÃ³n,Paquete TTL,360.00,57.60,417.60
20,LNHMBR,LEBR000287,CLAUDIA CASTILLO OROZCO,2,2,GestiÃ³n,Paquete TTL,360.00,57.60,417.60


In [39]:
%cd '/content/drive/My Drive/Medica_Vial_Experiment_1p1/output_files/Tables/'

/content/drive/My Drive/Medica_Vial_Experiment_1p1/output_files/Tables


In [40]:
!mkdir '/content/drive/My Drive/Medica_Vial_Experiment_1p1/output_files/Tables/{pk_for_files}'
%cd '/content/drive/My Drive/Medica_Vial_Experiment_1p1/output_files/Tables/{pk_for_files}'

/content/drive/My Drive/Medica_Vial_Experiment_1p1/output_files/Tables/31082020043737


In [41]:
def Turning_into_HTML ( df ) :
  Set_claves = list(set(list(df['Clave'])))
  for clave in Set_claves:
    table_by_clave = df[df.Clave.eq(clave)]
    Set_etapas = list(set(list(table_by_clave['Etapa'])))
    for etapa in Set_etapas:
      table_by_clave_and_set = table_by_clave[table_by_clave.Etapa.eq(etapa)]
      html_table = table_by_clave_and_set.to_html() #Especifica la ruta

      txt_file = open(f'table_{clave}_{etapa}.html', "w")
      txt_file.write(html_table)
      txt_file.close()

In [42]:
Turning_into_HTML( MV_first_output )

In [43]:
def Turning_into_CSV ( df ) :
  for clave in Set_claves:
    table_by_clave = df[df.Clave.eq(clave)]
    Set_etapas = list(set(list(table_by_clave['Etapa'])))
    for etapa in Set_etapas:
      table_by_clave_and_set = table_by_clave[table_by_clave.Etapa.eq(etapa)]
      csv_table = table_by_clave_and_set.to_csv(index = False) #Especifica la ruta

      txt_file = open(f'table_{clave}_{etapa}.csv', "w")
      txt_file.write(csv_table)
      txt_file.close()

In [44]:
Turning_into_CSV ( MV_first_output )

# Una vez que ya se crearon las tablas en html y en csv. Ya estamos listos para enviar los correos.

In [195]:
#list(MV_second_output_raw)
MV_second_output = MV_second_output_raw[['Clave','Unidad','Folio','Lesionado','Etapa', 'Entrega','Tipo', 'Lesion', 'Subtotal', 'IVA', 'Total', 'Consecutivo', 'Ones','RAZON SOCIAL', 'RFC', 'MAIL SOLICITUD DE FACTURAS' ]]

In [208]:
Set_claves

['LNHMBR', 'NLTHMA', 'TLXHSM', 'MORLUZ']

In [209]:
Set_etapas

[1, 2, 3]

In [224]:
#%cd '/content/drive/My Drive/Medica_Vial_Experiment_1p1/output_files/Tables/{pk_for_files}'
def Sending_E_mail ( df ) :
  Set_claves = list(set(list(df['Clave'])))
  for clave in Set_claves:
    df_by_clave = df[df.Clave.eq(clave)]
    Set_etapas = list(set(list(df_by_clave['Etapa'])))
    for etapa in Set_etapas:
      df_by_clave_and_set = df_by_clave[df_by_clave.Etapa.eq(etapa)]
      RFC = list(set(list(df_by_clave_and_set['RFC'])))[0]
      RazonSocial = list(set(list(df_by_clave_and_set['RAZON SOCIAL'])))[0]
      importeFactura = str(df_by_clave_and_set['Total'].sum())
      totalAtenciones = str(df_by_clave_and_set['Ones'].sum())
      #MV_first_output = MV_first_output[['Clave','Folio','Lesionado','Etapa', 'Entrega','Tipo', 'Lesion', 'Subtotal', 'IVA', 'Total']]
      jigsaw_falling_into_place = df_by_clave_and_set [['Clave','Folio','Lesionado','Etapa', 'Entrega','Tipo', 'Lesion', 'Subtotal', 'IVA', 'Total']]
      html_table = jigsaw_falling_into_place.to_html(index = False, justify = 'left')
      subject_date = datetime.today().strftime("%d_%m_%Y")
      subject = f'{clave}_{etapa}_{subject_date}'
      fechaProceso = datetime.today().strftime("%d/ %m/ %Y")
      etapa_str = str(etapa)

      body =  """<html>        
        <head> 
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>    
        <head> 	
            <style> 		
                table{ 		  
                border:1; 		
                    }  		
                table th { 		  
                color: #fff; 		  
                    background-color: blue; 		
                    } 
            </style>    
        </head>        
        <body>            
        <h1><span style="color:#353693; font-size:16pt;">$RazonSocial</span> </h1>
        <h2><span style="color:#353693; font-size:16pt;">$RFC</span></h2>
        <h4>Estimado Proveedor, </h4>    
        <h4>Por medio del presente solicito su apoyo para el env&#237;o de una factura global por un total de <span style="color:#353693; font-size:16pt;"> $ $importeFactura MXN </span>, as&#237; como sus respectivos archivos XML y PDF para cubrir el pago de <span style="color:#353693; font-size:16pt;"> $totalAtenciones </span> atenciones Etapas <span style="color:#353693; font-size:16pt;">[ $etapa_str ]</span>, adjunto relaci&#243;n actualizada al d&#237;a <span style="color:#353693; font-size:16pt;"> $fechaProceso </span> con la finalidad de programar su pago.
        <h4><span style='text-transform: uppercase;'>NOTA: SI EL METODO DE PAGO DE SU FACTURA ES PUE, DEBERA LLEGAR A MAS TARDAR EL DIA 25 DEL MES EN CURSO,  DE NO SER ASI TENDRIA QUE REFACTURARSE EN PPD Y GENERAR EL COMPLEMENTO DE PAGO DE LA MISMA. </span>   </h4>
        $html_table
        <p>
        <h4>Favor de no modificar la relaci&#243;n que se les env&#237;a, si tiene alguna duda estamos a sus &#243;rdenes.</h4>
        <h4>Le solicitamos anexar su factura (XML y PDF) dando respuesta a este mismo correo con la finalidad de poder procesas sus pagos de manera &#225;gil.</h4> 	
        <h4>Quedamos a la espera de su factura, agradeciendo su tiempo y atenci&#243;n a la misma.</h4>  	
        <img src="C:/Users/moral/Documents/BIGBOT/MedicaVialFinal/Repositorio/iniciales/FormatoBase/IMG-20200311-WA0008.jpg">   	
        <h4>Prueba aalonso@medicavial.com.mx / mvpagos@medicavial.com.mx <h4> 	
        </p>  
        </body>    
        </html>"""
      body_f = Template(body).safe_substitute(RFC = RFC, RazonSocial = RazonSocial, html_table = html_table,\
                                              importeFactura = importeFactura, totalAtenciones = totalAtenciones,\
                                              etapa_str = etapa_str, fechaProceso = fechaProceso)
      sender_email = 'facglobal@medicavial.com.mx'
      receiver_email_raw = list(set(list(df_by_clave_and_set['MAIL SOLICITUD DE FACTURAS'])))
      receiver_email = ','.join(map(str, receiver_email_raw))
      password = 'F4cgl0b4l2020'

      message = MIMEMultipart()
      message["From"] = sender_email
      message["To"] = receiver_email
      message["Subject"] = subject

      message.attach(MIMEText(body_f, "html"))

      filename = f'table_{clave}_{etapa}.csv'

      # Abre el archivo de la tabla y lo codifica
      with open(filename, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

      encoders.encode_base64(part)
      # Se agrega el nombre al adjunto de la tabla
      
      part.add_header("Content-Disposition",
      f"attachment; filename = {subject}.csv")

      # Se adjunta la tabla en el correo
      message.attach(part)
      text = message.as_string()
    
    # Ingreso al server y envio de correo
      smtp_ssl_host = 'mail.medicavial.com.mx'
      smtp_ssl_port = 465
      
      try:
        context = ssl.create_default_context()
        #with smtplib.SMTP_SSL("mail.medicavial.com.mx", 465, context=context) as server:
        #server.login(sender_email, password)
        #server.sendmail(sender_email, correoSend, text)
        server = smtplib.SMTP_SSL('mail.medicavial.com.mx:465')
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)
        server.quit()
        print(f'{clave}, {etapa}')
      except Exception as Ex:
        print(Ex)

In [225]:
MV_second_output = MV_second_output.reset_index( drop = True)

In [226]:
Sending_E_mail(MV_second_output)

LNHMBR, 2
LNHMBR, 3
NLTHMA, 1
TLXHSM, 2
TLXHSM, 3
MORLUZ, 1


In [227]:
def Looping_DF ( df ) :
  Set_claves = list(set(list(df['Clave'])))
  for clave in Set_claves:
    df_by_clave = df[df.Clave.eq(clave)]
    Set_etapas = list(set(list(df_by_clave['Etapa'])))
    for etapa in Set_etapas:
      print(f'{clave}_{etapa}')

In [228]:
Looping_DF(MV_second_output)

LNHMBR_2
LNHMBR_3
NLTHMA_1
TLXHSM_2
TLXHSM_3
MORLUZ_1


In [215]:
MV_second_output

,Clave,Unidad,Folio,Lesionado,Etapa,Entrega,Tipo,Lesion,Subtotal,IVA,Total,Consecutivo,Ones,RAZON SOCIAL,RFC,MAIL SOLICITUD DE FACTURAS
0,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001367,KARLA TAMARA CAMACHO BASORIA,1,1,Unica,Esguince Cervical Grado 2,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
1,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001369,ROMINA MABEL MENDIVIL LOPEZ,1,1,Unica,Heridas y Dermoabrasiones,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
2,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001380,EDUARDO TELLO CASTILLO,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
3,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001381,JESUS ORTIZ CERVANTES,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
4,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001383,ALICIA FRANCO RAMIREZ,1,1,Unica,Contusion Muscular y/o de Cavidad,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
5,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001384,LETICIA ARACELI NIÃ‘O ESTRADA,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
6,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001388,JUAN FRANCISCO VAZQUEZ CASTILLO,1,1,Unica,Contusion Craneana Leve,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
7,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001389,XIOMARA NICOLE PEREZ AMAYA,1,1,Leve,Leve,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
8,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001391,NICANDRO MENDEZ VAZQUEZ,1,1,Unica,Esguince Cervical Grado 1,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."
9,NLTHMA,NL Tamps-H. MexicoAmericano,TAMA001393,RICARDO MARTINEZ MEDINA,1,1,Multiple,Contusion Craneana Leve y Otras Lesiones,"1,155.00",92.40,"1,247.40",300,1,"Hospital Mexico Americano, S.C.",HMA980720GC2,"info@meluiscruz.com, luis.arturo.cruz.2@gmail...."


In [77]:
#Esta linea es solo un ensayo, no es necesario que se agrege al script final.
#Set_RFC = list(set(list(MV_second_output['RFC'])))
#Set_RFC

In [78]:
# Esta linea es solo un ensayo, no es necesario agregarla al script final.
#df_directorio_by_Clave = MV_second_output[MV_second_output.Clave.eq('NLTHMA')]
#df_directorio_by_Clave

In [79]:
# Esta linea es solo un ensayo, no es necesario agregarla al script final.
#df_directorio_by_Etapa = df_directorio_by_Clave[df_directorio_by_Clave.Etapa.eq(1)]
#df_directorio_by_Etapa

In [80]:
# Esta linea es solo un ensayo, no es necesario agregarla al script final.
#RFC = list(set(list(df_directorio_by_Etapa['RFC'])))
#RFC

In [81]:
# Esta linea es solo un ensayo, no es necesario agregarla al script final.
#RazonSocial = list(set(list(df_directorio_by_Etapa['RAZON SOCIAL'])))[0]
#RazonSocial

'Hospital Mexico Americano, S.C.'

In [57]:
# Esta linea es solo un ensayo, no es necesario agregarla al script final.
#importeFactura = df_directorio_by_Etapa['Total'].sum()

In [58]:
#importeFactura

13721.4

In [60]:
#totalAtenciones = df_directorio_by_Etapa['Ones'].sum()
#totalAtenciones

11

In [116]:
#fechaProceso = datetime.today().strftime("%d/%m/%Y")
#fechaProceso

'31/08/2020'